## Dermnet: Inception V3

Here I will use the Inception V3 model to classify the images. I will use the Dermnet dataset.

In [ ]:
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets
from torch.optim.lr_scheduler import StepLR
from torchvision import models
from torch import nn, optim
from tqdm.auto import tqdm

import torch.nn.functional as F
import torch
import yaml


# load config
with open('config/dermnet.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Check if GPU is available and if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data Loading

In [ ]:
# Define transforms
train_transforms = transforms.Compose([
    transforms.Resize((config['input_size'], config['input_size'])),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize((config['input_size'], config['input_size'])),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [ ]:
# Load datasets
train_ds = datasets.ImageFolder(config['train_dir'], transform=train_transforms)
test_ds = datasets.ImageFolder(config['test_dir'], transform=test_transforms)

# Create validation set
val_size = int(config['val_split'] * len(train_ds))
train_size = len(train_ds) - val_size
train_ds, val_ds = random_split(train_ds, [train_size, val_size])

# Create dataloaders
train_dl = DataLoader(
    dataset=train_ds,
    batch_size=config['batch_size'],
    shuffle=True
)
val_dl = DataLoader(
    dataset=val_ds,
    batch_size=config['batch_size'],
    shuffle=True
)
test_dl = DataLoader(
    dataset=test_ds,
    batch_size=config['batch_size'],
    shuffle=False
)

In [ ]:
for images, labels in train_dl:
    print('images.shape:', images.shape)
    print('labels.shape:', labels.shape)
    break

### Train/Test/Val

In [ ]:
# Load the pre-trained model
base_model = models.inception_v3(pretrained=True)

# Freeze the parameters
for param in base_model.parameters():
    param.requires_grad = False

# Get the number of input features for the classifier
num_features = base_model.fc.in_features 

# Define the classifier
class DermNetClassifier(nn.Module):
    """DermNet classifier"""

    def __init__(self):
        super(DermNetClassifier, self).__init__()
        
        self.fc1 = nn.Linear(num_features, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, config['num_classes'])

        self.dropout = nn.Dropout(0.3)

        self.bn1 = nn.BatchNorm1d(num_features=1024)
        self.bn2 = nn.BatchNorm1d(num_features=512)

    def forward(self, x):
        x = x.view(x.shape[0], -1)  # flatten tensor
        x = self.dropout( F.relu( self.bn1( self.fc1(x) ) ) )
        x = self.dropout( F.relu( self.bn2( self.fc2(x) ) ) )
        x = self.fc3(x)
        return x 
    
# Replace the classifier in the pre-trained model with our classifier
base_model.fc = DermNetClassifier()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(base_model.parameters(), lr=config['learning_rate'])

# Define a learning rate scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

# Move model to GPU if available
base_model.to(device)

In [ ]:
# store history
history = {
    'train_loss': [],
    'val_loss': [],
    'train_acc': [],
    'val_acc': []
}

# log training
writer = SummaryWriter('logs/dermnet')

for epoch in range(config['epochs']):
    train_loss = 0
    val_loss = 0
    accuracy = 0

    # Progress bar
    progress_bar = tqdm(total=len(train_dl.dataset), desc=f'Epoch {epoch + 1}')

    # Training
    base_model.train()
    for inputs, labels in train_dl:
        # Move tensors to GPU if CUDA is available
        inputs, labels = inputs.to(device), labels.to(device)

        # Clear the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs, _ = base_model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Add loss to the training set's running loss
        train_loss += loss.item() * inputs.size()[0]

        # Calculate accuracy
        top_p, top_class = outputs.topk(1, dim=1)
        equals = (top_class == labels.view(*top_class.shape))
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        # Log loss and accuracy
        history['train_loss'].append(loss.item()/len(inputs))
        history['train_acc'].append(accuracy/len(inputs))
        writer.add_scalar('Loss/train', loss.item()/len(inputs), epoch)
        writer.add_scalar('Accuracy/train', accuracy/len(inputs), epoch)

        # Update progress bar
        progress_bar.set_postfix({
            'train_loss': '{:.3f}'.format(loss.item()/len(inputs)),
            'train_acc': '{:.3f}'.format(accuracy/len(inputs))
        })
        progress_bar.update(inputs.shape[0])

    # End of epoch - evaluate on validation set
    with torch.no_grad():
        base_model.eval()
        for inputs, labels in val_dl:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = base_model(inputs)
            valloss = criterion(outputs, labels)
            val_loss += valloss.item() * inputs.size()[0]

            top_p, top_class = outputs.topk(1, dim=1)
            equals = (top_class == labels.view(*top_class.shape))
            val_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    # Log loss and accuracy
    history['val_loss'].append(val_loss/len(val_dl.dataset))
    history['val_acc'].append(val_accuracy/len(val_dl.dataset))
    writer.add_scalar('Loss/val', val_loss/len(val_dl.dataset), epoch)
    writer.add_scalar('Accuracy/val', val_accuracy/len(val_dl.dataset), epoch)

    # Update progress bar    
    progress_bar.set_postfix({
        'val_loss': '{:.3f}'.format(val_loss/len(val_dl.dataset)),
        'val_acc': '{:.3f}'.format(val_accuracy/len(val_dl.dataset))
    })
    progress_bar.close()
    
    # Step the learning rate scheduler
    scheduler.step()